In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

c:\Users\bpadmin\anaconda3\envs\tensorflow_cuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tensorflow_recommenders as tfrs

In [3]:
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "timestamp": x["timestamp"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

In [4]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [5]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

# need these to initialize timestamp embedding layers in future steps

timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

In [6]:
'''
this handles embedding user Identifiers and contextual data.
time stamp is used as the contexual information here.
using timestamp is 
'''

class UserModel(tf.keras.Model):
    def __init__(
        self, 
        use_timestamp,
        unique_user_ids, 
        timestamps, 
        timestamp_buckets):

        super().__init__()

        self.use_timestamp = use_timestamp
        self.unique_user_ids = unique_user_ids
        self.timestamp_buckets = timestamp_buckets
        self.timestamps = timestamps
        
        self.embed_user_id = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary = self.unique_user_ids,
                mask_token = None
            ),
            tf.keras.layers.Embedding(
                input_dim = len(self.unique_user_ids)+1,
                output_dim = 32
            )
        ])

        if self.use_timestamp:
            self.embed_timestamp = tf.keras.Sequential([
                tf.keras.layers.Discretization(
                    bin_boundaries = list(self.timestamp_buckets)
                ),

                tf.keras.layers.Embedding(
                    input_dim = len(list(self.timestamp_buckets))+1 ,
                    output_dim = 32
                )
            ])

            self.normalize_timestamp = tf.keras.layers.Normalization(
                axis = None #calcuate a scaler mean and variance 
            )
            self.normalize_timestamp.adapt(self.timestamps)

    
    def call(self, inputs):

        user_id, timestamp = inputs

        if self.use_timestamp:
            user_id_embed = self.embed_user_id(user_id)
            timestamp_embed = self.embed_timestamp(timestamp)
            norm_timestamp = tf.reshape(self.normalize_timestamp(timestamp), (-1,1)) #(-1,1) means first dimension to be infered

            return tf.concat([user_id_embed, timestamp_embed, norm_timestamp], axis = 1) #concatenate vertically
            
        return self.embed_user_id(user_id)

In [7]:
'''
this handles embedding item Identifiers and contextual data.
movie title itself is used as the contexual information here.
using timestamp is 
'''

class ItemModel(tf.keras.Model):
    def __init__(
        self,
        unique_movie_titles,
        ):
        super().__init__()

        self.max_tokens = 10000
        self.unique_movie_titles

        self.embed_item_id = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary = self.unique_movie_titles,
                mask_token =None
            ),
            tf.keras.layers.Embedding(
                input_dim = len(self.unique_movie_titles)+1,
                output_dim = 32
            )
        ])


        self.textvectorizer = tf.keras.layers.TextVectorization(
            max_tokens = self.max_tokens
        )

        self.embed_item_title = tf.keras.Sequential([
            self.textvectorizer,

            tf.keras.layers.Embedding(
                input_dim = self.max_tokens,
                output_dim = 32,
                mask_zero = True
            ),

            tf.keras.layers.GlobalAveragePooling1D() # reduces dimensionality to 1d (embedding layer embeddeds each word in a title one by one)
        ])

        self.textvectorizer.adapt(self.unique_movie_titles)
    
    def call(self, inputs):

        movie_title = inputs

        return tf.concat([
            self.embed_item_id(movie_title),
            self.embed_item_title(movie_title)
        ],
        axis = 1)
        
        # return self.embed_item_title(inputs['movie_title'])

In [8]:
class RankingModel(tf.keras.Model):

  def __init__(

    self,
    use_timestamp,
    unique_user_ids, 
    timestamps, 
    timestamp_buckets,
    unique_movie_titles
    ):
    
    super().__init__()

    # embedding_dimension = 32
    self.use_timestamp = use_timestamp
    self.unique_user_ids = unique_user_ids, 
    self.timestamps = timestamps, 
    self.timestamp_buckets = timestamp_buckets,
    self.unique_movie_titles = unique_movie_titles

    self.user_embeddings = UserModel(
      use_timestamp = self.use_timestamp,
      unique_user_ids = self.unique_user_ids, 
      timestamps = self.timestamps, 
      timestamp_buckets = self.timestamp_buckets
      )

    self.movie_embeddings = ItemModel(
      unique_movie_titles = self.unique_movie_titles
      )

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, timestamp, movie_title = inputs

    user_embedding = self.user_embeddings((user_id,timestamp))
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [9]:
test_user_model = UserModel(
    use_timestamp = True,
    unique_user_ids = unique_user_ids, 
    timestamps = timestamps, 
    timestamp_buckets = timestamp_buckets,
)

In [10]:
test_user_model((["42"],[886479771]))

<tf.Tensor: shape=(1, 65), dtype=float32, numpy=
array([[-0.04591095, -0.01999645, -0.04861705, -0.03927895,  0.02113768,
         0.03359554,  0.03974316,  0.00665708, -0.03207498, -0.03659819,
         0.0271732 ,  0.04544226, -0.02630298, -0.01789683,  0.0431136 ,
        -0.00808169,  0.00355117, -0.0261207 ,  0.01582832, -0.0086633 ,
         0.03367618, -0.03683843,  0.0203097 , -0.01870381, -0.04977077,
         0.03804914,  0.02514446, -0.04109829, -0.04488961,  0.03018678,
        -0.00188952,  0.03227441, -0.0071973 ,  0.04285849, -0.0406331 ,
         0.04734926,  0.03733401, -0.01190649, -0.04161644,  0.0249857 ,
        -0.02515309,  0.04760442, -0.03168219,  0.02971799, -0.0162881 ,
        -0.02007396, -0.00829492, -0.02891862, -0.02232406,  0.00701983,
         0.00155596,  0.00110019, -0.04234785,  0.04778567, -0.02763255,
         0.00759138,  0.04885993, -0.03452522, -0.02660987, -0.03732632,
         0.02734119, -0.0113823 , -0.01996914, -0.03777488,  0.55233073]],


In [ ]:
test_ranking_model = RankingModel(
    use_timestamp = True,
    unique_user_ids = unique_user_ids, 
    timestamps = timestamps, 
    timestamp_buckets = timestamp_buckets,
    unique_movie_titles = unique_movie_titles
    )

test_ranking_model((["42"],[886479771], ["One Flew Over the Cuckoo's Nest (1975)"]))

In [ ]:
class MovielensModel(tfrs.models.Model):

  def __init__(
    self,
    use_timestamp,
    unique_user_ids, 
    timestamps, 
    timestamp_buckets,
    unique_movie_titles
    ):

    super().__init__()

    self.use_timestamp = use_timestamp
    self.unique_user_ids = unique_user_ids, 
    self.timestamps = timestamps, 
    self.timestamp_buckets = timestamp_buckets,
    self.unique_movie_titles = unique_movie_titles

    self.ranking_model: tf.keras.Model = RankingModel(
      use_timestamp = self.use_timestamp,
      unique_user_ids = self.unique_user_ids, 
      timestamps = self.timestamps, 
      timestamp_buckets = self.timestamp_buckets,
      unique_movie_titles = self.unique_movie_titles
      )

    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features['timestamp'], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    return self.task(labels=labels, predictions=rating_predictions)

In [ ]:
model = MovielensModel(
    use_timestamp = True,
    unique_user_ids = unique_user_ids, 
    timestamps = timestamps, 
    timestamp_buckets = timestamp_buckets,
    unique_movie_titles = unique_movie_titles
    )
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
model.fit(cached_train, epochs=10)

Epoch 1/10


10/10 [==============================] - 0s 14ms/step - root_mean_squared_error: 1.1122 - loss: 1.2325 - regularization_loss: 0.0000e+00 - total_loss: 1.2325
Epoch 2/10
10/10 [==============================] - 0s 13ms/step - root_mean_squared_error: 1.1026 - loss: 1.2097 - regularization_loss: 0.0000e+00 - total_loss: 1.2097
Epoch 3/10
10/10 [==============================] - 0s 13ms/step - root_mean_squared_error: 1.0815 - loss: 1.1632 - regularization_loss: 0.0000e+00 - total_loss: 1.1632
Epoch 4/10
10/10 [==============================] - 0s 14ms/step - root_mean_squared_error: 1.0597 - loss: 1.1166 - regularization_loss: 0.0000e+00 - total_loss: 1.1166
Epoch 5/10
10/10 [==============================] - 0s 13ms/step - root_mean_squared_error: 1.0398 - loss: 1.0753 - regularization_loss: 0.0000e+00 - total_loss: 1.0753
Epoch 6/10
10/10 [==============================] - 0s 14ms/step - root_mean_squared_error: 1.0282 - loss: 1.0536 - regularization_loss: 0.0000e+00 - total_loss: 1.05

In [ ]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 25ms/step - root_mean_squared_error: 0.9756 - loss: 0.9494 - regularization_loss: 0.0000e+00 - total_loss: 0.9494


{'root_mean_squared_error': 0.9756286144256592,
 'loss': 0.9386831521987915,
 'regularization_loss': 0,
 'total_loss': 0.9386831521987915}

In [ ]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "timestamp": np.array([886479771]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
M*A*S*H (1970): [[3.9183755]]
Dances with Wolves (1990): [[3.6211972]]
Speed (1994): [[3.5145564]]
